In [ ]:
# !pip -qqq install gdown 


In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.metrics import classification_report
from tqdm import tqdm

In [2]:
# url = f"https://drive.google.com/uc?id={file_id}"
# output = "data.zip"  # change as needed
# gdown.download(url, output, quiet=False)

In [3]:
# import zipfile

# with zipfile.ZipFile("data.zip", 'r') as zip_ref:
#     zip_ref.extractall("")


In [4]:

freq = 50  # Hz
sampling_rate = 5000  # Hz
num_samples = 100  # samples per signal
duration = num_samples / sampling_rate  # 20 ms
t = np.linspace(0, duration, num_samples, endpoint=False)
num_signals = 1000
phases = np.linspace(0, 2 * np.pi, num_signals, endpoint=False)
signals = np.array([np.sin(2 * np.pi * freq * t + phase) for phase in phases])

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
sag = pd.read_csv("/kaggle/input/seed-power-quality-disturbance-dataset/XPQRS/Sag.csv",names = [f"time_step_{i}" for i in range(100)])
swell = pd.read_csv("/kaggle/input/seed-power-quality-disturbance-dataset/XPQRS/Swell.csv",names = [f"time_step_{i}" for i in range(100)])
harmonics = pd.read_csv("/kaggle/input/seed-power-quality-disturbance-dataset/XPQRS/Harmonics.csv",names = [f"time_step_{i}" for i in range(100)])
sag_harmonics = pd.read_csv("/kaggle/input/seed-power-quality-disturbance-dataset/XPQRS/Sag_with_Harmonics.csv",names = [f"time_step_{i}" for i in range(100)])
swell_harmonics = pd.read_csv("/kaggle/input/seed-power-quality-disturbance-dataset/XPQRS/Swell_with_Harmonics.csv",names = [f"time_step_{i}" for i in range(100)])
pure_sin = pd.DataFrame(signals,columns= [f"time_step_{i}" for i in range(100)])

In [7]:
sag["label"] = "sag"
swell["label"] = "swell"
harmonics["label"] = "harmonics"
sag_harmonics["label"] = "sag_harmonics"
swell_harmonics["label"] = "swell_harmonics"
pure_sin ["label"] = "sin"

In [8]:
data = pd.concat([sag,swell,pure_sin,harmonics,sag_harmonics,swell_harmonics])

In [9]:
data = data.drop_duplicates()

In [10]:
le = LabelEncoder()
le.fit_transform(data.label.unique())

array([1, 4, 3, 0, 2, 5])

In [11]:
class dataset(Dataset):
    def __init__(self,data):
        self.data = data.drop(columns = ["label"])
        self.Y_label = data[['label']]
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        return self.data.iloc[idx].values,le.transform(self.Y_label.iloc[idx].values)[0]        

In [12]:
dat = dataset(data)


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNLSTMModel(nn.Module):
    def __init__(self, num_classes=6):
        super(CNNLSTMModel, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(2)

        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(2)

        # LSTM takes input as (batch, seq_len, input_size)
        self.lstm = nn.LSTM(input_size=128, hidden_size=64, batch_first=True)

        self.fc1 = nn.Linear(64, 64)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Input shape: (batch, seq_len, channels)
        x = x.permute(0, 2, 1)  # → (batch, channels, seq_len)

        x = F.relu(self.bn1(self.conv1(x)))  # → (batch, 64, seq_len)
        x = self.pool1(x)                    # → (batch, 64, seq_len/2)

        x = F.relu(self.bn2(self.conv2(x)))  # → (batch, 128, seq_len/2)
        x = self.pool2(x)                    # → (batch, 128, seq_len/4)

        x = x.permute(0, 2, 1)  # → (batch, seq_len/4, 128) for LSTM
        x, _ = self.lstm(x)     # → (batch, seq_len/4, 64)

        x = x[:, -1, :]         # Use output from last time step
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [37]:
class CNN1DModel(nn.Module):
    def __init__(self, num_classes=6):
        super(CNN1DModel, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Similar to GlobalAveragePooling1D

        self.fc1 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.15)
        self.fc3 = nn.Linear(64,num_classes)
        self.fc2 = nn.Linear(128, 64)
    
    def forward(self, x):
        # Input: (batch_size, 100, 9)
        x = x.permute(0, 2, 1)  # Convert to (batch_size, channels=9, time_steps=100)
        
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = self.global_avg_pool(x)  # shape: (batch_size, 256, 1)
        x = x.view(x.size(0), -1)    # shape: (batch_size, 256)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x  # Or use raw logits and `CrossEntropyLoss`


In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ANN_TimeSeries(nn.Module):
    def __init__(self, input_timesteps=100, input_features=1, num_classes=6):
        super(ANN_TimeSeries, self).__init__()
        
        self.input_size = input_timesteps * input_features  # Flattened size
        
        self.fc1 = nn.Linear(self.input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        # x shape: (B, T, C)
        x = x.view(x.size(0), -1)  # Flatten to (B, T*C)
        
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x  # raw logits, use with nn.CrossEntropyLoss()


In [46]:
model = ANN_TimeSeries(num_classes=6)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay = 1e-4)
criterion = nn.CrossEntropyLoss()
# scheduler = ReduceLROnPlateau(optimizer, mode = "min", factor=0.5,patience = 5)
model = model.to(device)

In [29]:
train_dataset,test_dataset = train_test_split(dat,test_size = 0.2)

In [30]:

train_loader = DataLoader(train_dataset,batch_size = 32,shuffle = True)
val_loader = DataLoader(test_dataset,batch_size = 32,shuffle = True)

In [39]:
best_val_loss = float("inf")

In [47]:
for epoch in range(80):
    model.train()
    total_loss = 0
    print(f"\nEpoch {epoch+1}")
    
    for X_batch, y_batch in tqdm(train_loader, desc="Training"):
        X_batch,y_batch = X_batch.float().to(device),y_batch.to(device)
        X_batch = X_batch.unsqueeze(2)
        optimizer.zero_grad()
        out = model(X_batch)
        loss = criterion(out, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train Loss: {avg_train_loss:.4f}")
    
    # Evaluation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for X_batch,y_batch in tqdm(val_loader, desc="Evaluating"):
            X_batch,y_batch = X_batch.float().to(device),y_batch.to(device)
            X_batch = X_batch.unsqueeze(2)
            out = model(X_batch,)
            loss = criterion(out, y_batch)
            val_loss += loss.item()
            pred = out.argmax(dim=1)
            correct += (pred == y_batch).sum().item()
            total += y_batch.size(0)

    avg_val_loss = val_loss / len(val_loader)
    acc = correct / total
    print(f"Val Loss: {avg_val_loss:.4f}")
    print(f"Test Accuracy: {acc:.4f}")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_model_norm.pth')
        torch.save(optimizer.state_dict(),"best_state_norm.pth")
    if epoch%10 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] /= 2
    


Epoch 1


Training: 100%|██████████| 150/150 [00:00<00:00, 376.24it/s]


Train Loss: 1.1811


Evaluating: 100%|██████████| 1200/1200 [00:00<00:00, 2377.53it/s]


Val Loss: 0.8958
Test Accuracy: 0.6542

Epoch 2


...

Train Loss: 0.2636


Evaluating: 100%|██████████| 1200/1200 [00:00<00:00, 2397.24it/s]


Val Loss: 0.1838
Test Accuracy: 0.9425

Epoch 36


Training:   0%|          | 0/150 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [34]:
val_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [48]:
y_pred = []
y_true = []
model.eval()
with torch.no_grad():
    for X_batch,y_batch in tqdm(val_loader, desc="Evaluating"):
        X_batch,y_batch = X_batch.float().to(device),y_batch.to(device)
        X_batch = X_batch.unsqueeze(2)
        out = model(X_batch)
        pred = out.argmax(dim=1)
        y_pred.append(int(pred[0]))
        y_true.append(int(y_batch[0]))
        


Evaluating: 100%|██████████| 1200/1200 [00:00<00:00, 2670.86it/s]


In [86]:
le.classes_

array(['harmonics', 'sag', 'sag_harmonics', 'sin', 'swell',
       'swell_harmonics'], dtype=object)

In [41]:
print(classification_report(y_pred,y_true))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       216
           1       1.00      0.99      1.00       203
           2       0.98      1.00      0.99       199
           3       1.00      0.97      0.99       178
           4       1.00      0.99      0.99       214
           5       0.97      1.00      0.99       190

    accuracy                           0.99      1200
   macro avg       0.99      0.99      0.99      1200
weighted avg       0.99      0.99      0.99      1200



In [36]:
print(classification_report(y_pred,y_true))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       216
           1       1.00      0.99      1.00       203
           2       0.98      1.00      0.99       199
           3       1.00      0.97      0.99       178
           4       1.00      0.99      0.99       214
           5       0.97      1.00      0.99       190

    accuracy                           0.99      1200
   macro avg       0.99      0.99      0.99      1200
weighted avg       0.99      0.99      0.99      1200



In [49]:
print(classification_report(y_pred,y_true))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       218
           1       0.96      0.93      0.94       206
           2       0.85      1.00      0.92       174
           3       1.00      0.86      0.93       201
           4       0.99      0.90      0.94       232
           5       0.85      0.98      0.91       169

    accuracy                           0.94      1200
   macro avg       0.94      0.94      0.94      1200
weighted avg       0.95      0.94      0.94      1200



In [ ]:
Metadata:
Fundamental Frequency: 50 Hz
Sampling Rate: 5 kHz
Number of Classes: 17
Signals per Class: 1000
Length of Each Signal (samples): 100
Length of Each Signal (time): 20 ms
